In [1]:
# 환경설정

!pip3 install transformers
!pip3 install torch
!pip3 install konlpy
!pip3 install jamo
!pip3 install sentence-transformers
!pip3 install flask
!pip3 install flask-ngrok

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 35.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 41.7 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 1.2 MB 11.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.w

In [2]:
# 시험데이터셋 수집

!wget https://raw.githubusercontent.com/aifactory-team/hanryubank/main/warehouse.zip
!unzip -q warehouse.zip

--2021-11-23 14:57:52--  https://raw.githubusercontent.com/aifactory-team/hanryubank/main/warehouse.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15349 (15K) [application/zip]
Saving to: ‘warehouse.zip’

warehouse.zip       100%[===================>]  14.99K  --.-KB/s    in 0s      

2021-11-23 14:57:53 (55.5 MB/s) - ‘warehouse.zip’ saved [15349/15349]



In [3]:
# 모듈 설정

from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
qa = pipeline("question-answering", tokenizer=tokenizer, model=model)
qs = SentenceTransformer('bespin-global/klue-korsts-roberta-base-sentence-embedding') # qs question similarity

import konlpy
okt = konlpy.tag.Okt()

from jamo import h2j, j2hcj

vowels = ['ㅣ', 'ㅔ', 'ㅐ', 'ㅏ', 'ㅜ', 'ㅗ', 'ㅓ', 'ㅡ', 'ㅟ', 'ㅚ', 'ㅑ', 'ㅕ', 'ㅛ', 'ㅠ', 'ㅒ', 'ㅖ', 'ㅘ', 'ㅝ', 'ㅙ', 'ㅞ', 'ㅢ']

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.3M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/587 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

In [4]:
# 입력된 질문(query)에 해당하는 유사한 질문을 찾고, 그 유사한 질문에 해당하는 지문을 검색하는 함수

def get_context_and_question(query:str, ts:float):

    # 데이터베이스 로딩
    df = pd.read_csv('dataset_test.csv')
    df = df.iloc[:24, :8]

    # 유사도
    sim = []
    # 데이터베이스에서 사용자 질의와 유사한 질문 획득
    for db_question in df['Q1 (질문1)']:
      
      # 문장에 대해 사전 학습된 문맥 정보를 활용한 임베딩 결과로 인코딩
      embedding1 = qs.encode(db_question, convert_to_tensor=True)
      embedding2 = qs.encode(query, convert_to_tensor=True)
      # compute similarity scores of two embeddings
      cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
      sim.append(cosine_scores.item())
    
    # 유사 질문에 해당하는 지문과 질문 획득
    context = df.loc[sim.index(max(sim))]["Context (지문)"]
    question = df.loc[sim.index(max(sim))]["Q1 (질문1)"]

    return context, question

In [5]:
# 지문과 질문을 입력하여 정답을 얻는 함수

def get_answer_with_context(context, question):
    result = qa({"question": question, "context": context})
    score = result["score"]
    start = result["start"]
    end = result["end"]
    answer = result["answer"]
    return answer, start, end, score

In [6]:
# 해답의 어미를 바꾸는 함수

def modify_answer(before_answer):

    document = before_answer
    
    clean_words = []

    for word in okt.pos(document, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_words.append(word[0])
    
    last_char = j2hcj(h2j(clean_words[-1]))[-1]

    if last_char in vowels:
        clean_words.append('라네')
    else:
        clean_words.append('이라네')

    after_answer = ''.join(clean_words)

    return after_answer

In [7]:
# 질문으로 답 얻기

def print_answer(query):

    # 질문에 해당하는 데이터베이스에 저장된 유사 질문 및 지문 획득
    context, question = get_context_and_question(query, 0.5)

    # 지문과 질문을 입력해서 정답을 얻음
    answer, start, end, score = get_answer_with_context(context, question)

    # 출력
    print("query: " + query)
    print("question: " + question)
    print("predict: {0} ({1},{2},{3})".format(answer, start, end, score))
    print("service: " + modify_answer(answer))

In [8]:
print_answer('훈민정음을 창제한 이유')
print_answer('훈민정음을 왜 창제했지?')
print_answer('훈민정음을 만든 이유는?')
print_answer('세종대왕이 훈민정음을 창제한 이유는?')
print_answer('세종대왕이 훈민정음을 왜 창제했을까?')
print_answer('세종대왕은 왜 훈민정음을 창제했지?')

print_answer('조세에 관하여 무엇을 시행했는가')
print_answer('세종대왕이 조세와 관련해서 뭘 시행했지?')
print_answer('조세랑 관련해서 뭘 했지?')

query: 훈민정음을 창제한 이유
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 훈민정음을 왜 창제했지?
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 훈민정음을 만든 이유는?
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 세종대왕이 훈민정음을 창제한 이유는?
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 세종대왕이 훈민정음을 왜 창제했을까?
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 세종대왕은 왜 훈민정음을 창제했지?
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 조세에 관하여 무엇을 시행했는가
question: 조세에 관하여 무엇을 시행했는가
predict: 여론 조사를 (30,36,0.9997721314430237)
service: 여론조사라네
query: 세종대왕이 조세와 관련해서 뭘 시행했지?
question: 조세에 관하여 무엇을 시행했는가
predict: 여론 조사를 (30,36,0.9997721314430237)
service: 여론조사라네
query: 조세랑 관련해서 뭘 했지?
question: 조세에 관하여 무엇을 시행했는가
predict: 여론 조사를 (30,36,0.9997721314430237)
service: 여론조사라네


In [9]:
from flask import *
from flask_ngrok import run_with_ngrok
import flask

app = Flask(__name__)

@app.route('/')
@app.route('/index')
def index():
    return flask.render_template('index.html')

@app.route('/hello')
def hello():
  return 'Hello World'

@app.route('/answer', methods=['GET', 'POST'])
def answer():
    if request.method == 'POST':
        query = str(request.form['query'])

        # 질문에 해당하는 데이터베이스에 저장된 유사 질문 및 지문 획득
        context, question = get_context_and_question(query, 0.5)

        # 지문과 질문을 입력해서 정답을 얻음
        db_answer, start, end, score = get_answer_with_context(context, question)

        service_answer = modify_answer(db_answer)

    return flask.render_template('answer.html', service_answer = service_answer, db_answer = db_answer, score=score, start=start, end=end, query=query, question=question, context=context)


In [10]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_debugpy.py", line 59, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_variable_inspector.py", line 28, in run
    globals().clear()
TypeError: 'module' object is not callable



 * Running on http://359f-34-66-5-239.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 14:59:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 14:59:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 14:59:49] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 15:00:01] "GET /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 15:00:25] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 15:00:33] "GET /index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 15:01:03] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 15:01:31] "POST /answer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Nov/2021 15:02:22] "GET /index HTTP/1.1" 200 -
